In [1]:
from dotenv import load_dotenv
from langchain.messages import HumanMessage, AIMessage
from langchain.agents import create_agent
from langchain.chat_models import  init_chat_model
from pydantic import BaseModel

In [2]:
load_dotenv()

True

# Basic prompting

In [3]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
agent = create_agent(model=model)


In [4]:
question = HumanMessage(content="What's the capital of the Moon?")

In [5]:
response = agent.invoke({"messages": [question]})
response["messages"][-1].content

'That\'s a fun question!\n\nThe Moon doesn\'t have a capital because it\'s not a country or a political entity, and it doesn\'t have any cities, governments, or permanent inhabitants (yet!).\n\nIf humans ever colonize the Moon and establish independent settlements, then they might decide on a capital for their lunar society. For now, it\'s just a celestial body.\n\nHowever, if we were to pick a historically significant spot, **Tranquility Base** (the landing site of Apollo 11) might be a good symbolic candidate for a "first landing point" or "birthplace of human lunar exploration."'

In [6]:
system_prompt = "You are a science fiction writer, create a capital city at the users requests."
scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt,
)

response = scifi_agent.invoke({"messages": [question]})
print(response["messages"][-1].content)

That's an excellent question! In the nascent but thriving future of humanity's lunar expansion, the capital of the Moon is known as **Artemis City**.

Nestled deep within the vast, ancient lava tube systems bordering the **Malapert Mountain region** near the Moon's South Pole, Artemis City is a marvel of subterranean engineering. Its location was chosen for the unparalleled natural shielding provided by the lava tubes against radiation and micrometeorites, as well as its strategic proximity to the abundant water ice reserves in nearby permanently shadowed craters.

**Key Features of Artemis City:**

*   **Subterranean Metropolis:** The bulk of Artemis City is carved out of and built within several massive, interconnected lava tubes. These vast caverns have been sealed, pressurized, and terraformed, housing residential sectors, sprawling hydroponic farms (glowing with specialized full-spectrum lighting), research labs, industrial facilities, and cultural centers.
*   **The Nexus Dome:**

# Few show exaples

In [7]:
system_prompt = """
You are a science fiction writer, create a space capital at thr users requests.

User: What ish the capital of mars?
Scifi Writer: Masialis

User: What is the capital of Venus?
Scifi Writer: Venusovia

"""

scifi_agent = create_agent(model=model, system_prompt=system_prompt)

response = agent.invoke({"messages": [question]})

response["messages"][-1].content

'The Moon does not have a capital.\n\nA capital city is the administrative center of a country or region, and the Moon is a celestial body, not a political entity with a government or established population that would have one.'

# Structured prompts

In [8]:
system_prompt = """
You are a science fiction writer, create a space capital city at the users request.

Please keep to the below structure.

Name: The name of the capital city

Location: Where it is based

Vibe: 2-4 word to describe its vibe

Economy: Main industries

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt,
)

response = agent.invoke({"messages": [question]})
response["messages"][-1].content

'That\'s a fun question that sparks the imagination!\n\nHowever, the Moon does not have a capital. Here\'s why:\n\n*   **No Countries or Governments:** The Moon is a celestial body, not a sovereign nation. There are no countries, governments, or political structures on the Moon.\n*   **No Cities or Permanent Settlements (Yet!):** While humans have visited the Moon, there are no permanent cities or settlements that would necessitate a capital city.\n\nPerhaps one day in the distant future, if humanity establishes colonies on the Moon, they might designate a "capital" for their lunar settlement!'

# Structured output

In [9]:
class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the user request.",
    response_format=CapitalInfo
)

response = agent.invoke({"messages": [question]})


In [10]:
response

{'messages': [HumanMessage(content="What's the capital of the Moon?", additional_kwargs={}, response_metadata={}, id='44564127-8d40-42d5-be5c-eeff9aed64f0'),
  AIMessage(content='{"name": "Selene Prime", "location": "Within the perpetually shadowed depths of Shackleton Crater at the lunar South Pole, protected by massive translucent domes and subterranean levels carved from regolith.", "vibe": "A breathtaking fusion of bio-luminescent hydroponic gardens and sleek, chrome-alloy architecture, buzzing with the quiet hum of advanced technology. It feels like a vibrant, self-contained ecosystem, a testament to human ingenuity and a hub of constant innovation.", "economy": "Primarily driven by Helium-3 mining and processing for interstellar fusion drives, advanced astrobotics manufacturing, zero-gravity luxury tourism, and cutting-edge xenobiological research."}', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_

In [11]:
response["structured_response"]

CapitalInfo(name='Selene Prime', location='Within the perpetually shadowed depths of Shackleton Crater at the lunar South Pole, protected by massive translucent domes and subterranean levels carved from regolith.', vibe='A breathtaking fusion of bio-luminescent hydroponic gardens and sleek, chrome-alloy architecture, buzzing with the quiet hum of advanced technology. It feels like a vibrant, self-contained ecosystem, a testament to human ingenuity and a hub of constant innovation.', economy='Primarily driven by Helium-3 mining and processing for interstellar fusion drives, advanced astrobotics manufacturing, zero-gravity luxury tourism, and cutting-edge xenobiological research.')

In [12]:
response["structured_response"].name

'Selene Prime'

In [13]:
capital_info = response["structured_response"]

capital_name = capital_info.name
capital_location = capital_info.location

print(f"{capital_name} is a city loacted at {capital_location}")

Selene Prime is a city loacted at Within the perpetually shadowed depths of Shackleton Crater at the lunar South Pole, protected by massive translucent domes and subterranean levels carved from regolith.


# Examples

In [14]:
system_prompt = """
You have got best knowladge about Hollywood actors. Give the best feedback based on user request.
"""

actor_agent = create_agent(model=model, system_prompt=system_prompt)
response = actor_agent.invoke({"messages":[HumanMessage(content="Who is Tom Cruise?")]})
response["messages"][-1].content

'Tom Cruise is an American actor and producer, widely regarded as one of the most iconic and successful movie stars in Hollywood history.\n\nHere\'s a breakdown of what makes him such a significant figure:\n\n1.  **Blockbuster Star Power:** He has an unparalleled ability to consistently deliver massive box office hits. His films have grossed billions worldwide, making him one of the highest-grossing actors of all time.\n\n2.  **Action Icon:** Cruise is synonymous with action cinema, particularly known for performing his own incredibly dangerous stunts. This dedication is a huge part of his brand, most famously showcased in the **Mission: Impossible** franchise, where he plays the impossible-task-taking Ethan Hunt. Other notable action roles include *Top Gun* (and its immensely successful sequel *Top Gun: Maverick*), *Minority Report*, and *Edge of Tomorrow*.\n\n3.  **Versatile Actor (Early Career):** While now known primarily for action, Cruise established himself in a wide range of ge

In [15]:
system_prompt = """
You have got best knowladge about Hollywood actors. Give the best feedback based on user request.

Please keep to the below structure.

Name: Actor's full name

Location: Where was born

All movies: All movies count

The best movies: The best 10 movies

"""

actor_agent = create_agent(model=model, system_prompt=system_prompt)
response = actor_agent.invoke({"messages":[HumanMessage(content="Who is Tom Cruise?")]})
response["messages"][-1].content

'Name: Tom Cruise\n\nLocation: Syracuse, New York, USA\n\nAll movies: 46\n\nThe best movies:\n1.  Top Gun (1986)\n2.  Rain Man (1988)\n3.  Born on the Fourth of July (1989)\n4.  A Few Good Men (1992)\n5.  Mission: Impossible (1996)\n6.  Jerry Maguire (1996)\n7.  Magnolia (1999)\n8.  Minority Report (2002)\n9.  Collateral (2004)\n10. Top Gun: Maverick (2022)'

In [19]:
class ActorInfo(BaseModel):
    name: str
    location: str
    all_movies: str
    best_movies: str

actor_agent = create_agent(
    model=model, 
    system_prompt="You have got best knowladge about Hollywood actors. Give the best feedback based on user request.",
    response_format=ActorInfo
)
response = actor_agent.invoke({"messages":[HumanMessage(content="Who is Tom Cruise?")]})
response["messages"][-1].content

'{\n  "name": "Tom Cruise",\n  "location": "Syracuse, New York, USA",\n  "all_movies": "Tom Cruise has an extensive filmography spanning decades, with roles in action, drama, and science fiction films. Some of his notable films include \'Risky Business\', \'Top Gun\', \'Rain Man\', \'Born on the Fourth of July\', \'A Few Good Men\', \'Mission: Impossible\' series, \'Jerry Maguire\', \'Magnolia\', \'Minority Report\', \'The Last Samurai\', \'War of the Worlds\', \'Valkyrie\', \'Oblivion\', \'Edge of Tomorrow\', and \'Top Gun: Maverick\'.",\n  "best_movies": "Some of Tom Cruise\'s most acclaimed and iconic films include \'Top Gun\' and its sequel \'Top Gun: Maverick\', \'Mission: Impossible\' series (especially \'Fallout\'), \'Rain Man\', \'Born on the Fourth of July\', \'Jerry Maguire\', \'A Few Good Men\', \'Magnolia\', and \'Edge of Tomorrow\'."\n}'

In [17]:
response["structured_response"]

ActorInfo(name='Tom Cruise', location='Syracuse, New York, USA', all_movies='A prolific actor known for a wide range of films across various genres including action, drama, sci-fi, and thrillers, with a career spanning over four decades.', best_movies='Top Gun, Rain Man, Mission: Impossible series, Jerry Maguire, A Few Good Men, Minority Report, Top Gun: Maverick')

In [21]:
actor_info = response["structured_response"]
actor_place = actor_info.location
actor_name = actor_info.name

print(f"{actor_name} was born in {actor_place}")

Tom Cruise was born in Syracuse, New York, USA
